In [42]:
from __future__ import print_function, division
import os
import keras
from keras.layers import Rescaling


class DataLoader():
    def __init__(self, dataset_name=None, img_res=(128, 128)):
        self.dataset_name = dataset_name
        self.img_res = img_res

    # needs to be modified to be "negative" and "positive" for the two classes, not capitalized
    def load_batch(self, train_N="negative", train_P="positive", batch_size=16, is_testing=False):
        
        subdir = "validation" if is_testing else "train"

        # Normalize: (lambda x: x / 127.5 - 1) can be applied as part of a preprocessing layer
        normalization_layer = Rescaling(1./127.5, offset=-1)
        
        # Load negative (class 0) images only
        data_negative = keras.utils.image_dataset_from_directory(
            f"../data/{subdir}/{train_N}",
            labels=None,
            color_mode='grayscale',
            batch_size=batch_size,
            image_size=self.img_res,
            shuffle=True
        ).map(lambda x: normalization_layer(x))

        data_positive = keras.utils.image_dataset_from_directory(
            f"../data/{subdir}/{train_P}",
            labels=None,
            color_mode='grayscale',
            batch_size=batch_size,
            image_size=self.img_res,
            shuffle=True
        ).map(lambda x: normalization_layer(x))

        # endless loop so we can use the maximum
        n_batches = max(len(data_negative), len(data_positive))

        for normal, pneumo, _ in zip(data_negative, data_positive, range(n_batches)):
            yield normal, pneumo

    def load_single(self, path):
        img = keras.preprocessing.image.load_img(path, color_mode="grayscale", target_size=self.img_res)
        # print("Performing preprocessing; ... incorecct!, now done in model layer")
        # x = keras.preprocessing.image.img_to_array(img) / 127.5 - 1
        x = keras.preprocessing.image.img_to_array(img)
        return x.reshape((1, self.img_res[0], self.img_res[1], 1))

    def save_single(self, x, path):
        # Rescale images 0 - 1
        x = 0.5 * x + 0.5
        keras.preprocessing.image.save_img(path, x)


In [43]:
from keras.models import load_model
def load_classifier(path):
    classifier = load_model(path, compile=True)
    classifier.summary()
    return classifier

In [44]:
data_loader = DataLoader(img_res=(512, 512))
img = data_loader.load_single("../data/train/negative/ffcc35e8-6fe8-42df-9605-9fedf57240d1.png")
classifier = load_classifier("classifier2.keras")
classification = classifier.predict(img)

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 512, 512, 1)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ rescaling_1 (Rescaling)         │ (None, 512, 512, 1)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 126, 126, 96)   │        11,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_9 (Activation)       │ (None, 126, 126, 96)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 63, 63, 96)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_8           │ (None, 63, 63, 96)     │           384 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_6 (Conv2D)               │ (None, 53, 53, 256)    │     2,973,952 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_10 (Activation)      │ (None, 53, 53, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 26, 26, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_9           │ (None, 26, 26, 256)    │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_7 (Conv2D)               │ (None, 24, 24, 384)    │       885,120 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_11 (Activation)      │ (None, 24, 24, 384)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_10          │ (None, 24, 24, 384)    │         1,536 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_8 (Conv2D)               │ (None, 22, 22, 384)    │     1,327,488 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_12 (Activation)      │ (None, 22, 22, 384)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_11          │ (None, 22, 22, 384)    │         1,536 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_9 (Conv2D)               │ (None, 20, 20, 256)    │       884,992 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_13 (Activation)      │ (None, 20, 20, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_5 (MaxPooling2D)  │ (None, 10, 10, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_12          │ (None, 10, 10, 256)    │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 25600)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 4096)           │   104,861,696 │
├─────────────────────────────────┼────────────────────────┼─────────────

 Total params: 263,713,958 (1005.99 MB)

 Trainable params: 131,846,410 (502.95 MB)

 Non-trainable params: 21,136 (82.56 KB)

 Optimizer params: 131,846,412 (502.95 MB)

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 736ms/step


In [45]:
classification

array([[0.95867175, 0.04132823]], dtype=float32)